# Convex Optimization Project

**Pouya Lahabi 400109834**

**Aylin Rasteh 400104964**

In this notebook we implemented some of the optimizations in transfer learning paper and used some of the ideas from [Multi-Task Feature Learning paper](https://proceedings.neurips.cc/paper_files/paper/2006/file/0afa92fc0f8a9cf051bf2961b06ac56b-Paper.pdf) (one of the main paper refrences).

## Abstract

A major assumption in many machine learning and data mining algorithms is that the training and future data must be in the same feature space and have the same distribution. However, in many real-world applications, this assumption may not hold. In recent years, transfer learning has emerged as a new learning framework to address this problem. This survey focuses on categorizing and reviewing the current progress on transfer learning for classification, regression, and clustering problems. In this project we will implement parts 3.2.1 and 3.3 of the original [paper](https://www-edlab.cs.umass.edu/cs689/reading/transfer-learning.pdf).

## Notation

We begin by introducing our notation. We let $\mathbb{R}$ be the set of real numbers and $\mathbb{R}^+$ ($\mathbb{R}^{++}$) the subset of non-negative (positive) ones. Let $T$ be the number of tasks and define $\mathbb{R}_T := \{1, \ldots, T\}$. For each task $t \in \mathbb{R}_T$, we are given $m$ input/output examples $(x_{t1}, y_{t1}), \ldots, (x_{tm}, y_{tm}) \in \mathbb{R}^d \times \mathbb{R}$. Based on this data, we wish to estimate $T$ functions $f_t : \mathbb{R}^d \rightarrow \mathbb{R}$, $t \in \mathbb{R}_T$, which approximate well the data and are statistically predictive.

Let $a_i \in \mathbb{R}^T$ and $a_j \in \mathbb{R}^d$ be the $i$-th row and the $j$-th column of $A$ respectively. For every $r, p \geq 1$, we define
$
\|A\|_{r,p} := \left( \sum_{i=1}^{d} \|a_i\|_p^r \right)^{\frac{1}{p}}.
$

If $w, u \in \mathbb{R}^d$, we define $\langle w, u \rangle := \sum_{i=1}^{d} w_i u_i$, the standard inner product in $\mathbb{R}^d$. For every $p \geq 1$, we define the $p$-norm of vector $w$ as $\|w\|_p := \left( \sum_{i=1}^{d} |w_i|^p \right)^{\frac{1}{p}}$. If $A$ is a $d \times T$ matrix, we denote by
$
\|A\|_{r,p} := \left( \sum_{i=1}^{d} \|a_i\|_p^r \right)^{\frac{1}{p}}.
$

We denote by $S_d$ the set of $d \times d$ real symmetric matrices and by $S_d^+$ the subset of positive semidefinite ones. If $D$ is a $d \times d$ matrix, we define $\text{trace}(D) := \sum_{i=1}^{d} D_{ii}$. If $X$ is a $p \times q$ real matrix, $\text{range}(X)$ denotes the set $\{ x \in \mathbb{R}^p : x = Xz, \text{ for some } z \in \mathbb{R}^q \}$. We let $O_d$ be the set of $d \times d$ orthogonal matrices. Finally, $D^+$ denotes the pseudoinverse of a matrix $D$.

## 3.2.1 Supervised Feature Construction
In the inductive transfer learning setting, the common features can be learned by solving an optimization problem, given as:
\begin{align*}
\underset{A, U}{arg\;min}& \quad \sum \limits_{t \in \{T, S\}} \sum \limits_{i=1}^{n_t}L\left(y_{t_i}, \langle a_t, U^T x_{t_i} \rangle\right) + \gamma ||A||^2_{2,1} \\
s.t.& \quad U \in \mathbb{O}^d
\end{align*}

This method learns a low-dimensional representation which is shared across the task.

Solving the given task can be difficult because of two main reasons:
1) Although the problem is convex in each of the variables $A$ and $U$ but the main problem is non-convex.
2) $||A||_{2,1}$ is nonsmooth which makes it more difficult to opimize

To this end, for every $W \in \mathbb{R}^{d\times T}$ and $D \in \mathbb{S}^d_+$, we define the function

$$
R(W, D) = \sum\limits_{t=1}^T \sum\limits_{i=1}^{m} L(y_{t_i}, \langle w_t, x_{t_i}\rangle) + \gamma \sum\limits_{t=1}^T\langle w_t, D^\dagger w_t \rangle
$$

\begin{align*}
min& \qquad R(W, D) \\
s.t.& \qquad W \in \mathbb{R}^{d\times T} \\
& \qquad D \in \mathbb{S}_+^d\\
& \qquad trace(D) \leq 1 \\
& \qquad range(W) \subseteq range(D)
\end{align*}
That is, $(\hat{A}, \hat{U})$ is an optimal solution for our first formulation if and only if $(\hat{W}, \hat{D}) = (\hat{U}\hat{A}, \hat{U}Diag(\hat{\lambda})\hat{U}^T)$ is an optimal solution for the equivalent convex problem if and only if
$$
\hat{\lambda}_i := \frac{||\hat{a}^i||_2}{||\hat{A}_{2,1}||}
$$

### **Proof**.
Let $(W = UA)$ and $(D = U\text{Diag}(\frac{||a^i||_2}{||A||_{2,1}})U^T)$. Then $||a^i||_2 = ||W^Tu_i||_2$ and hence

\begin{align*}
\sum\limits_{t=1}^{T} \langle w_t, D^\dagger w_t \rangle = \mathrm{trace}(W^T D^\dagger W) = ||A||_{2,1} \mathrm{trace}(W^TU\mathrm{Diag}(||W^Tu_i||_2)^\dagger U^T W) = \\
||A||_{2, 1} \mathrm{trace}(\sum\limits_{i=1}^d(||W^T u_i||_2)^\dagger W^T u_iu_i^T W) = ||A||_{2,1}\sum\limits_{i=1}^d ||W^Tu_i||_2 = ||A||^2_{2,1}
\end{align*}
Therefore, $( \min_{W,D} R(W, D) \leq \min_{A,U} \epsilon(A, U) )$. Conversely, let $(D = U\text{Diag}(\lambda)U^\dagger$). Then

\begin{align*}
\sum_{t=1}^{T} \langle w_t, D^\dagger w_t \rangle &= \text{trace}(W^\dagger U\text{Diag}(\lambda+i)U^\dagger W) \\
&= \text{trace}\left(\text{Diag}(\lambda+i)AA^\dagger\right) \geq ||A||_{2,1}^2
\end{align*}

Where $\epsilon(A, U) = \sum\limits_{t=1}^{T}\sum\limits_{i=1}^{m}L(y_{t_i}, \langle a_t, U^T x_{t_i} \rangle) + \gamma ||A||^2_{2,1}$


In [ ]:
import cvxpy as cp
import numpy as np

In [142]:
d = 10
T = 5
m = 100
gamma = 0.1

# Define the variables
coefficients = cp.Variable((d, T))
W = cp.Variable((d, T))
D = cp.Variable((d, d), symmetric=True)
D = cp.atoms.affine.wraps.psd_wrap(D)
U = cp.Variable((d, d), symmetric = True)
U = cp.atoms.affine.wraps.psd_wrap(U)

X = np.random.rand(d, m, T)
Y = np.random.randint(2, size=(m, T))


objective = 0
tmp = []
for t in range(T):
    for i in range(m):
        objective += cp.norm2(Y[i, t] - cp.sum(cp.multiply(W[:, t], X[:, i, t])))
    objective += gamma * cp.quad_form(W[:, t], U)
    # objective += gamma * W[:, t].T  @ U @ W[:, t]
    # objective += gamma * cp.sum(cp.multiply(W[:, t].T, W[:, t].T))
    # objective += cp.sum(cp.multiply(W[:, t].T, cp.matmul(U, W[:, t])))
    # cp.sum(cp.multiply(x, y))
    # objective += gamma * cp.norm(W[:, t], 2)
    # for i in range(d):
    #     objective += W[i][t] * W[i][t]
    # for j in range(d):
    #     for k in range(d):
    #         objective += gamma * U[j, k] * W[j, t]* W[k, t]
    
constraints = []
constraints.append(D @ U == np.eye(d))
constraints.append(cp.trace(D) <= 1)
constraints.append(D >> 0)
constraints.append(U >> 0)
for t in range(T):
    constraints.append(W[:, t] == cp.sum([D[:, i] * coefficients[i, t] for i in range(d)]))

problem = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)

problem.solve()

# optimal_W = W.value
optimal_D = D.value

Exception: At least one argument to quad_form must be non-variable.

We tried different methods to implement $\gamma \sum\limits_{t=1}^T\langle w_t, D^\dagger w_t \rangle$ and some of the constraints, but we faced different kinds of errors in the process of multilying two cvx vector variables

## Relaxation

when the matrix $U$ is not learned and we set $U = I_{d\times d}$, our problem, computes a common set of variables across the tasks. That is, we have the following convex optimization problem

\begin{align*}
min \qquad \sum_{t=1}^T\sum_{i=1}^m L(y_{t_i}, \langle a_t, x_{t_i} \rangle) + \gamma ||A||_{2,1}^2
\end{align*}

In [140]:
## Relaxtion

d = 10
T = 5
m = 100 
gamma = 0.1

A = cp.Variable((d, T))

X = np.random.rand(d, m, T)
Y = np.random.randint(2, size=(m, T))

objective = 0
for t in range(T):
    for i in range(m):
        objective += cp.norm2(Y[i, t] - cp.multiply(A[:, t], X[:, i, t]))
    objective += gamma * cp.norm2(A[:, t])**2
    
problem = cp.Problem(objective=cp.Minimize(objective))

problem.solve()

712.5946939520976

## 3.3. Transferring knowledge of parameters

In inductive transfer learning,

$w_S = w_0 + v_S$

$w_T = w_0 + v_T $

where $ w_S$ and $ w_T $ are parameters of the SVMs for the source task and the target learning task, respectively. $ w_0 $ is a common parameter, while $ v_S $ and $ v_T $ are specific parameters for the source task and the target task, respectively. By assuming $ f_t = w_t \cdot x $ to be a hyperplane for task $ t $, an extension of SVMs to the multitask learning case can be written as follows:
$f_t = w_t \cdot x$

\begin{align*}
\underset{w_0, v_t, \xi_{t_i}}{min} & \qquad J(w_0, v_t, \xi_{t_i}) = \sum\limits_{t \in \{S, T\}}\sum\limits_{i=1}^{n_t} \xi_{t_i} + \frac{\lambda_1}{2} \sum\limits_{t \in \{S, T\}} \|v_t\|^2 + \lambda_2 \|w_0\|^2 \\
s.t. & \qquad y_{t_i} (w_0 + v_t) x_{t_i} \geq q - \xi_{t_i},\quad\xi_{t_i} \geq 0,\; i \in \{1,2,\cdots,n_t\}\;\mathrm{and}\;t \in \{S, T\}
\end{align*}

In [143]:
d = 10
T = 5
m = 100
lambda_1 = 0.1
lambda_2 = 0.1

X = np.random.rand(d, m, T)
Y = np.random.randint(2, size=(m, T))

xi = cp.Variable((T, m))

w_0 = cp.Variable(d)
v = cp.Variable((T, d))

objective = 0
for t in range(T):
    for i in range(m):
        objective += xi[t, i]
    objective += lambda_1 / 2 * cp.norm2(v[t, :]) ** 2
objective += lambda_2 * cp.norm2(w_0) ** 2

constraints = []
for t in range(T):
    for i in range(m):
        constraints.append(xi[t, i] >= 0)
        constraints.append(Y[i, t] * (w_0 + v[t, :]) @ X[:,i, t] >= 1 - xi[t, i])

problem = cp.Problem(objective=cp.Minimize(objective), constraints=constraints)
problem.solve()

246.08815257879678